In [1]:
# First 9 weeks & rewards in 19 Audience only 

import pandas as pd
import datetime
import numpy as np
import os
import glob
import gc

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/AOV_analysis_by_audience'

In [2]:
audience_list_2019Q3="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/"
audience_list_2019Q3=glob.glob(audience_list_2019Q3+"*.csv")
len(audience_list_2019Q3)


audience_list_2019Q3_C=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="C"]
audience_list_2019Q3_T=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="T"]
print(len(audience_list_2019Q3_C),len(audience_list_2019Q3_T))

set_C_2019Q3=set()
for file in audience_list_2019Q3_C:
    df=pd.read_csv(file,dtype=str,usecols=["customer_id_hashed"])
    set_C_2019Q3=set_C_2019Q3.union(set(df['customer_id_hashed'].tolist()))
print(len(set_C_2019Q3))


df_audience_2019Q3=pd.DataFrame()

for file in audience_list_2019Q3_T:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df=df[~df['customer_id_hashed'].isin(set_C_2019Q3)]
    df_audience_2019Q3=df_audience_2019Q3.append(df)
df_audience_2019Q3=df_audience_2019Q3.drop_duplicates()

print(df_audience_2019Q3.shape," | ",df_audience_2019Q3['customer_id_hashed'].nunique())

25 25
542702
(21810554, 2)  |  21810554


In [3]:
# Rolling up removing the quadrant
df_audience_total_count_original=df_audience_2019Q3.groupby("segment")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment'].str.replace("Quadrant I_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant II_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant III_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant IV_","")
print(df_audience_2019Q3.shape)

df_audience_total_count=df_audience_2019Q3.groupby("segment_2")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})

del df_audience_2019Q3['segment']
df_audience_2019Q3=df_audience_2019Q3.rename(columns={"segment_2":"segment"})

print(df_audience_total_count.shape)
df_audience_total_count

(21810554, 3)
(7, 2)


,segment_2,ids_total
0,T_P_H_2019Q3_RFM,4937981
1,T_P_L_2019Q3_RFM,3391401
2,T_P_M_2019Q3_RFM,3515875
3,T_S_H_2019Q3_RFM,1694485
4,T_S_L_2019Q3_RFM,1422640
5,T_S_M_2019Q3_RFM,1361003
6,T_lapsed_18_48_PS_RFM,5487169


In [4]:
# New ids

new_sign_ups_since_2019Q3_end=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_since_2019Q3_end=new_sign_ups_since_2019Q3_end[new_sign_ups_since_2019Q3_end['sign_up_date']>="2018-08-05"] # First of 2018Q3
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_since_2019Q3_end=master_file_gap.append(new_sign_ups_since_2019Q3_end).drop_duplicates()
del master_file_gap
###

all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("/MediaStorm_")[1][:10]>"2019-05-01"]

df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df_master_all_later_after_201906=df_master_all_later_after_201906.append(df)

new_sign_ups_since_2019Q3_end=new_sign_ups_since_2019Q3_end.append(df_master_all_later_after_201906).drop_duplicates()
del df_master_all_later_after_201906


gc.collect()
print("new_sign_ups_since_2019Q3_end.shape",new_sign_ups_since_2019Q3_end.shape)
print("new_sign_ups_since_2019Q3_end['customer_id_hashed'].nunique()",new_sign_ups_since_2019Q3_end['customer_id_hashed'].nunique())

dup_ids=new_sign_ups_since_2019Q3_end[new_sign_ups_since_2019Q3_end['customer_id_hashed'].duplicated()]['customer_id_hashed'].tolist()
df=new_sign_ups_since_2019Q3_end[new_sign_ups_since_2019Q3_end['customer_id_hashed'].isin(dup_ids)]

new_sign_ups_since_2019Q3_end=new_sign_ups_since_2019Q3_end.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False]).drop_duplicates("customer_id_hashed")
set_new_signed_ids=set(new_sign_ups_since_2019Q3_end['customer_id_hashed'].tolist())
del new_sign_ups_since_2019Q3_end
gc.collect()

new_sign_ups_since_2019Q3_end.shape (8135521, 2)
new_sign_ups_since_2019Q3_end['customer_id_hashed'].nunique() 8135510


35

In [5]:
print(len(set_new_signed_ids))

8135510


# Without Product Information

In [6]:
sales_file_2019Q3=list(recursive_file_gen("/home/jian/BigLots/"))
sales_file_2019Q3=[x for x in sales_file_2019Q3 if "Daily" in x and x[-4:]==".txt" and "2019-" in x]
sales_file_2019Q3=[x for x in sales_file_2019Q3 if x.split("/MediaStorm_")[1][:10]>="2019-08-05" and x.split("/MediaStorm_")[1][:10]<="2019-10-10"]
# date only >=
len(sales_file_2019Q3)


9

In [7]:
# First 9 weeks only
sales_file_2018Q3=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
sales_file_2018Q3=[x for x in sales_file_2018Q3 if x.split("MediaStormDailySalesHistory")[1][:8]>"20180804" and x.split("MediaStormDailySalesHistory")[1][:8]<="20181008"]
len(sales_file_2018Q3)

9

In [8]:
df_agg_POS_data_2018Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2018Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed'],how="outer")
    df_agg_POS_data_2018Q3_rewards=df_agg_POS_data_2018Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-10-16 10:49:00.107108 0 (1190721, 6)
2019-10-16 10:54:18.755599 1 (2414440, 6)
2019-10-16 10:58:36.923315 2 (3646503, 6)
2019-10-16 11:02:44.688973 3 (4894015, 6)
2019-10-16 11:06:51.178192 4 (6144836, 6)
2019-10-16 11:10:49.382260 5 (7315812, 6)
2019-10-16 11:14:31.728694 6 (8459424, 6)
2019-10-16 11:19:32.599606 7 (9898381, 6)
2019-10-16 11:24:01.685510 8 (11287537, 6)


96

In [9]:
df_agg_POS_data_2019Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2019Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed'],how="outer")
    df_agg_POS_data_2019Q3_rewards=df_agg_POS_data_2019Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2019Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-10-16 11:27:53.484493 0 (1312276, 6)
2019-10-16 11:32:29.080256 1 (2641439, 6)
2019-10-16 11:36:40.753387 2 (3993285, 6)
2019-10-16 11:42:44.671540 3 (5407040, 6)
2019-10-16 11:46:16.082892 4 (6796632, 6)
2019-10-16 11:49:42.963747 5 (8103909, 6)
2019-10-16 11:53:18.741436 6 (9423221, 6)
2019-10-16 11:57:04.437314 7 (11014408, 6)
2019-10-16 12:04:06.075018 8 (12541623, 6)


96

In [10]:
def week_end_dt(x):
    y=datetime.datetime.strptime(x,"%Y-%m-%d").date()
    if y.weekday()==6:
        y=y+datetime.timedelta(days=6)
    else:
        y=y+datetime.timedelta(days=5-y.weekday())
    return y

In [11]:
df_week_end_dt=df_agg_POS_data_2019Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt_2=df_agg_POS_data_2018Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt=df_week_end_dt.append(df_week_end_dt_2)
del df_week_end_dt_2
df_week_end_dt['week_end_dt']=df_week_end_dt['transaction_dt'].apply(week_end_dt)


In [12]:
df_agg_POS_data_2018Q3_rewards=pd.merge(df_agg_POS_data_2018Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")
df_agg_POS_data_2019Q3_rewards=pd.merge(df_agg_POS_data_2019Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")


In [13]:
write_folder="./output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [14]:
df_agg_POS_data_2019Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2019Q3_rewards.csv",index=False)
df_agg_POS_data_2018Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2018Q3_rewards.csv",index=False)

In [15]:
# Registered before 2018Q3 only
print(df_agg_POS_data_2019Q3_rewards.shape)
print(df_agg_POS_data_2018Q3_rewards.shape)

df_agg_POS_data_2019Q3_rewards=df_agg_POS_data_2019Q3_rewards[~df_agg_POS_data_2019Q3_rewards['customer_id_hashed'].isin(set_new_signed_ids)]
df_agg_POS_data_2018Q3_rewards=df_agg_POS_data_2018Q3_rewards[~df_agg_POS_data_2018Q3_rewards['customer_id_hashed'].isin(set_new_signed_ids)]


(12541623, 7)
(11287537, 7)


In [16]:
df_2019Q3_by_id=df_agg_POS_data_2019Q3_rewards.groupby(['customer_id_hashed'])['sales','units','trans'].sum().reset_index()
df_2018Q3_by_id=df_agg_POS_data_2018Q3_rewards.groupby(['customer_id_hashed'])['sales','units','trans'].sum().reset_index()

# df_2019Q3_by_id_week=df_agg_POS_data_2019Q3_rewards.groupby(['customer_id_hashed','week_end_dt'])['sales','units','trans'].sum().reset_index()
# df_2018Q3_by_id_week=df_agg_POS_data_2018Q3_rewards.groupby(['customer_id_hashed','week_end_dt'])['sales','units','trans'].sum().reset_index()

print(df_agg_POS_data_2019Q3_rewards.shape)
print(df_agg_POS_data_2018Q3_rewards.shape)

print(df_2019Q3_by_id.shape)
print(df_2018Q3_by_id.shape)

# print(df_2019Q3_by_id_week.shape)
# print(df_2018Q3_by_id_week.shape)

(9525575, 7)
(10209645, 7)
(5185319, 4)
(5563584, 4)


In [17]:
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")
print(df_2019Q3_by_id.shape)
print(df_2018Q3_by_id.shape)

# df_2019Q3_by_id_week=pd.merge(df_2019Q3_by_id_week,df_audience_2019Q3,on="customer_id_hashed",how="left")
# df_2018Q3_by_id_week=pd.merge(df_2018Q3_by_id_week,df_audience_2019Q3,on="customer_id_hashed",how="left")
# print(df_2019Q3_by_id_week.shape)
# print(df_2018Q3_by_id_week.shape)

df_2019Q3_by_id['segment']=df_2019Q3_by_id['segment'].fillna("balance")
df_2018Q3_by_id['segment']=df_2018Q3_by_id['segment'].fillna("balance")
# df_2018Q3_by_id_week['segment']=df_2018Q3_by_id_week['segment'].fillna("balance")
# df_2019Q3_by_id_week['segment']=df_2019Q3_by_id_week['segment'].fillna("balance")

(5185319, 5)
(5563584, 5)


In [18]:
agg_function={"customer_id_hashed":"nunique","sales":"sum","units":"sum","trans":"sum"}

df_output_19=df_2019Q3_by_id.groupby("segment")['customer_id_hashed','sales','units','trans'].agg(agg_function).reset_index()
df_output_19=df_output_19.rename(columns={"customer_id_hashed":"count_shoppers"})
for col in df_output_19.columns.tolist()[1:]:
    df_output_19=df_output_19.rename(columns={col:col+"_2019"})
    
df_output_18=df_2018Q3_by_id.groupby("segment")['customer_id_hashed','sales','units','trans'].agg(agg_function).reset_index()
df_output_18=df_output_18.rename(columns={"customer_id_hashed":"count_shoppers"})
for col in df_output_18.columns.tolist()[1:]:
    df_output_18=df_output_18.rename(columns={col:col+"_2018"})


In [19]:
df_output=pd.merge(df_output_18,df_output_19,on="segment")
df_audience_total_count=df_audience_total_count.rename(columns={"segment_2":"segment"})
df_output=pd.merge(df_output,df_audience_total_count,on="segment",how="left")


cols=["segment","ids_total","sales_2019","sales_2018","units_2019","units_2018","trans_2019","trans_2018","count_shoppers_2019","count_shoppers_2018"]
df_output=df_output[cols]


In [50]:
del df_output['AOV_2019']
del df_output['AOV_2018']

del df_output['SalesPerShopper_2019']
del df_output['SalesPerShopper_2018']

In [51]:
df_output['AOV_2019']=df_output['sales_2019']/df_output['trans_2019']
df_output['AOV_2018']=df_output['sales_2018']/df_output['trans_2018']

df_output['UnitPerTrans_2019']=df_output['units_2019']/df_output['trans_2019']
df_output['UnitPerTrans_2018']=df_output['units_2018']/df_output['trans_2018']


df_output['SalesPerShopper_2019']=df_output['sales_2019']/df_output['count_shoppers_2019']
df_output['SalesPerShopper_2018']=df_output['sales_2018']/df_output['count_shoppers_2018']


df_output['TransPerShopper_2019']=df_output['trans_2019']/df_output['count_shoppers_2019']
df_output['TransPerShopper_2018']=df_output['trans_2018']/df_output['count_shoppers_2018']


df_output['UnitPerShopper_2019']=df_output['units_2019']/df_output['count_shoppers_2019']
df_output['UnitPerShopper_2018']=df_output['units_2018']/df_output['count_shoppers_2018']


In [20]:
del df_agg_POS_data_2019Q3_rewards
del df_agg_POS_data_2018Q3_rewards
del df_2019Q3_by_id
del df_2018Q3_by_id
gc.collect()

177

In [52]:
df_output

,segment,ids_total,sales_2019,sales_2018,units_2019,units_2018,trans_2019,trans_2018,count_shoppers_2019,count_shoppers_2018,AOV_2019,AOV_2018,UnitPerTrans_2019,UnitPerTrans_2018,SalesPerShopper_2019,SalesPerShopper_2018,TransPerShopper_2019,TransPerShopper_2018,UnitPerShopper_2019,UnitPerShopper_2018
0,T_P_H_2019Q3_RFM,4937981.0,1.513964e+08,1.568991e+08,34638733,37269915,4905401,4969709,1986048,2019594,30.863206,31.571079,7.061346,7.499416,76.229981,77.688425,2.469931,2.460747,17.441035,18.454162
1,T_P_L_2019Q3_RFM,3391401.0,1.821562e+07,1.682016e+07,2668876,2927391,434169,472419,295866,400984,41.955147,35.604336,6.147090,6.196599,61.567143,41.947222,1.467451,1.178149,9.020557,7.300518
2,T_P_M_2019Q3_RFM,3515875.0,4.196397e+07,6.856101e+07,7302789,12562336,1105461,1837912,677739,1020133,37.960610,37.303748,6.606103,6.835113,61.917602,67.207909,1.631101,1.801640,10.775223,12.314410
3,T_S_H_2019Q3_RFM,1694485.0,4.719755e+07,4.917203e+07,10499334,11410684,1381345,1404277,612535,623518,34.167824,35.015904,7.600805,8.125665,77.052826,78.862245,2.255128,2.252184,17.140790,18.300489
4,T_S_L_2019Q3_RFM,1422640.0,7.619965e+06,7.329770e+06,1100409,1263769,166913,189787,115060,161457,45.652316,38.621031,6.592710,6.658881,66.226013,45.397658,1.450661,1.175465,9.563784,7.827279
5,T_S_M_2019Q3_RFM,1361003.0,1.592390e+07,2.651176e+07,2744227,4751179,385567,640924,241158,366565,41.299956,41.364904,7.117380,7.413015,66.030985,72.324853,1.598815,1.748459,11.379374,12.961355
6,T_lapsed_18_48_PS_RFM,5487169.0,1.673176e+07,6.014869e+04,2172591,13193,345018,1510,239155,1502,48.495337,39.833570,6.297037,8.737086,69.962009,40.045732,1.442654,1.005326,9.084447,8.783622
7,balance,NaN,7.577442e+07,7.224081e+07,12815730,12745081,1850182,1708605,1017758,969831,40.955117,42.280578,6.926740,7.459349,74.452296,74.488037,1.817900,1.761755,12.592119,13.141548


# By product division

In [22]:
Product_Taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190901-135942-034.txt",
                              sep="|",dtype=str)
Product_Taxonomy=Product_Taxonomy[['class_code_id','department_id','division_id']].drop_duplicates()
Product_Taxonomy.shape,Product_Taxonomy['class_code_id'].nunique()
Product_Taxonomy_name=Product_Taxonomy.copy()
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding="iso-8859-1")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,division_name,on="division_id",how="left")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,department_name,on="department_id",how="left")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,class_name,on="class_code_id",how="left")
Product_Taxonomy_name.shape

(462, 6)

In [23]:
df_agg_POS_data_2018Q3_rewards_division=pd.DataFrame()
df_agg_POS_data_2018Q3_rewards_department=pd.DataFrame()

i_counter=0
for file in sales_file_2018Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[~df['customer_id_hashed'].isin(set_new_signed_ids)]  # remove the new sign ups

    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,Product_Taxonomy_name,on='class_code_id',how="left")
    
    # Division level
    df_sales_division=df.groupby(['location_id','transaction_dt','customer_id_hashed','division_id','division_desc'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales_division=df_sales_division.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans_division=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','division_desc']].drop_duplicates()
    df_trans_division['trans']=1
    df_trans_division=df_trans_division.groupby(['location_id','transaction_dt','customer_id_hashed','division_id','division_desc'])['trans'].sum().to_frame().reset_index()

    df_division=pd.merge(df_sales_division,df_trans_division,on=["location_id",'transaction_dt','customer_id_hashed','division_id','division_desc'],how="outer")
    
    # Department level
    df_sales_department=df.groupby(['location_id','transaction_dt','customer_id_hashed','department_id','department_desc'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales_department=df_sales_department.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans_department=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','department_id','department_desc']].drop_duplicates()
    df_trans_department['trans']=1
    df_trans_department=df_trans_department.groupby(['location_id','transaction_dt','customer_id_hashed','department_id','department_desc'])['trans'].sum().to_frame().reset_index()

    df_department=pd.merge(df_sales_department,df_trans_department,on=["location_id",'transaction_dt','customer_id_hashed','department_id','department_desc'],how="outer")
    
    df_agg_POS_data_2018Q3_rewards_division=df_agg_POS_data_2018Q3_rewards_division.append(df_division)
    df_agg_POS_data_2018Q3_rewards_department=df_agg_POS_data_2018Q3_rewards_department.append(df_department)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards_division.shape)
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards_department.shape)
    i_counter+=1
gc.collect()

# new sign ups already removed

2019-10-16 12:18:49.481519 0 (2368093, 8)
2019-10-16 12:18:49.482471 0 (3681525, 8)
2019-10-16 12:22:09.174021 1 (4800824, 8)
2019-10-16 12:22:09.175693 1 (7508965, 8)
2019-10-16 12:25:22.205561 2 (7222526, 8)
2019-10-16 12:25:22.206809 2 (11274812, 8)
2019-10-16 12:28:35.997732 3 (9633372, 8)
2019-10-16 12:28:35.998981 3 (15062157, 8)
2019-10-16 12:31:50.731988 4 (12024247, 8)
2019-10-16 12:31:50.734111 4 (18820857, 8)
2019-10-16 12:34:59.920778 5 (14252854, 8)
2019-10-16 12:34:59.921724 5 (22361656, 8)
2019-10-16 12:38:06.008600 6 (16393496, 8)
2019-10-16 12:38:06.009869 6 (25736266, 8)
2019-10-16 12:42:44.351633 7 (19163356, 8)
2019-10-16 12:42:44.353380 7 (30304048, 8)
2019-10-16 12:50:30.660817 8 (21914810, 8)
2019-10-16 12:50:30.663279 8 (34787212, 8)


63

In [24]:
df_agg_POS_data_2019Q3_rewards_division=pd.DataFrame()
df_agg_POS_data_2019Q3_rewards_department=pd.DataFrame()

i_counter=0
for file in sales_file_2019Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,Product_Taxonomy_name,on='class_code_id',how="left")
    df=df[~df['customer_id_hashed'].isin(set_new_signed_ids)] # remove the new sign ups

    # Division level
    df_sales_division=df.groupby(['location_id','transaction_dt','customer_id_hashed','division_id','division_desc'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales_division=df_sales_division.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans_division=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','division_desc']].drop_duplicates()
    df_trans_division['trans']=1
    df_trans_division=df_trans_division.groupby(['location_id','transaction_dt','customer_id_hashed','division_id','division_desc'])['trans'].sum().to_frame().reset_index()

    df_division=pd.merge(df_sales_division,df_trans_division,on=["location_id",'transaction_dt','customer_id_hashed','division_id','division_desc'],how="outer")
    
    # Department level
    df_sales_department=df.groupby(['location_id','transaction_dt','customer_id_hashed','department_id','department_desc'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales_department=df_sales_department.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans_department=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','department_id','department_desc']].drop_duplicates()
    df_trans_department['trans']=1
    df_trans_department=df_trans_department.groupby(['location_id','transaction_dt','customer_id_hashed','department_id','department_desc'])['trans'].sum().to_frame().reset_index()

    df_department=pd.merge(df_sales_department,df_trans_department,on=["location_id",'transaction_dt','customer_id_hashed','department_id','department_desc'],how="outer")
    
    df_agg_POS_data_2019Q3_rewards_division=df_agg_POS_data_2019Q3_rewards_division.append(df_division)
    df_agg_POS_data_2019Q3_rewards_department=df_agg_POS_data_2019Q3_rewards_department.append(df_department)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2019Q3_rewards_division.shape)
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2019Q3_rewards_department.shape)
    i_counter+=1
gc.collect()

# new sign ups already removed

2019-10-16 12:53:42.680412 0 (2123544, 8)
2019-10-16 12:53:42.681319 0 (3308276, 8)
2019-10-16 12:57:01.499442 1 (4283340, 8)
2019-10-16 12:57:01.500730 1 (6716449, 8)
2019-10-16 13:00:33.103081 2 (6472265, 8)
2019-10-16 13:00:33.105549 2 (10134954, 8)
2019-10-16 13:04:10.645719 3 (8722899, 8)
2019-10-16 13:04:10.648118 3 (13709263, 8)
2019-10-16 13:06:54.022004 4 (10910252, 8)
2019-10-16 13:06:54.022960 4 (17165321, 8)
2019-10-16 13:10:11.056293 5 (12964240, 8)
2019-10-16 13:10:11.057550 5 (20421793, 8)
2019-10-16 13:13:34.904444 6 (15031591, 8)
2019-10-16 13:13:34.905745 6 (23684280, 8)
2019-10-16 13:17:51.823215 7 (17555830, 8)
2019-10-16 13:17:51.824952 7 (27788032, 8)
2019-10-16 13:21:55.584563 8 (20032754, 8)
2019-10-16 13:21:55.585803 8 (31794731, 8)


63

In [25]:
del df
del df_department
del df_division
gc.collect()

104

In [26]:
print(datetime.datetime.now())
df_agg_POS_data_2019Q3_rewards_division.head(2)

2019-10-16 13:21:59.919705


,location_id,transaction_dt,customer_id_hashed,division_id,division_desc,units,sales,trans
0,1,2019-08-04,023978948616cd96f8ddf41b5122cd9291b89d0b47675f...,1,FOOD,2,9.50,1
1,1,2019-08-04,0304353c3725d559df8e954aeb75ed1b6e40f14812835c...,1,FOOD,3,5.09,1


In [27]:
# Rolling up to id, division level
df_agg_POS_data_2019Q3_rewards_division=df_agg_POS_data_2019Q3_rewards_division.groupby(['customer_id_hashed','division_id','division_desc'])['sales','units','trans'].sum().reset_index()
df_agg_POS_data_2018Q3_rewards_division=df_agg_POS_data_2018Q3_rewards_division.groupby(['customer_id_hashed','division_id','division_desc'])['sales','units','trans'].sum().reset_index()

# Rolling up to id, department level
df_agg_POS_data_2019Q3_rewards_department=df_agg_POS_data_2019Q3_rewards_department.groupby(['customer_id_hashed','department_id','department_desc'])['sales','units','trans'].sum().reset_index()
df_agg_POS_data_2018Q3_rewards_department=df_agg_POS_data_2018Q3_rewards_department.groupby(['customer_id_hashed','department_id','department_desc'])['sales','units','trans'].sum().reset_index()

gc.collect()

206

In [28]:
df_agg_POS_data_2019Q3_rewards_division=pd.merge(df_agg_POS_data_2019Q3_rewards_division,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_agg_POS_data_2018Q3_rewards_division=pd.merge(df_agg_POS_data_2018Q3_rewards_division,df_audience_2019Q3,on="customer_id_hashed",how="left")

df_agg_POS_data_2019Q3_rewards_department=pd.merge(df_agg_POS_data_2019Q3_rewards_department,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_agg_POS_data_2018Q3_rewards_department=pd.merge(df_agg_POS_data_2018Q3_rewards_department,df_audience_2019Q3,on="customer_id_hashed",how="left")


df_agg_POS_data_2019Q3_rewards_division['segment']=df_agg_POS_data_2019Q3_rewards_division['segment'].fillna("balance")
df_agg_POS_data_2018Q3_rewards_division['segment']=df_agg_POS_data_2018Q3_rewards_division['segment'].fillna("balance")

df_agg_POS_data_2019Q3_rewards_department['segment']=df_agg_POS_data_2019Q3_rewards_department['segment'].fillna("balance")
df_agg_POS_data_2018Q3_rewards_department['segment']=df_agg_POS_data_2018Q3_rewards_department['segment'].fillna("balance")


print(df_agg_POS_data_2019Q3_rewards_division.shape)
print(df_agg_POS_data_2018Q3_rewards_division.shape)

print(df_agg_POS_data_2019Q3_rewards_department.shape)
print(df_agg_POS_data_2018Q3_rewards_department.shape)

(13619612, 7)
(14967745, 7)
(24957198, 7)
(27377028, 7)


In [67]:
df_agg_POS_data_2018Q3_rewards_division.head(2)

,customer_id_hashed,division_id,division_desc,sales,units,trans,segment
0,000008374518306e87838f387eedc47d49a447707dc6da...,2,CONSUMABLES,4.88,1,1,T_P_H_2019Q3_RFM
1,000008374518306e87838f387eedc47d49a447707dc6da...,5,HARDHOME,4.50,1,1,T_P_H_2019Q3_RFM


(13619612, 7)

In [31]:
df_agg_POS_data_2019Q3_rewards_department.head(2)

,customer_id_hashed,department_id,department_desc,sales,units,trans,segment
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,170,HOME ORGANIZATION,3.8,1,1,balance
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,110,BEVGROCERY,3.0,3,1,T_P_H_2019Q3_RFM


In [54]:
df_output_seg_division_19=df_agg_POS_data_2019Q3_rewards_division.groupby(["segment","division_id","division_desc"])['sales','units','trans','customer_id_hashed'].agg(agg_function).reset_index()
df_output_seg_division_19_long=df_output_seg_division_19.rename(columns={"customer_id_hashed":"shopper_count"})
df_output_seg_division_19_wide=df_output_seg_division_19_long.pivot_table(index="segment",columns="division_desc",values=['sales','units','trans','shopper_count'])
df_output_seg_division_19_wide=df_output_seg_division_19_wide.reset_index()

new_cols=[]
for col in df_output_seg_division_19_wide.columns.tolist():
    if col[1]=="":
        new_cols.append(col[0])
    else:
        new_cols.append("Q3_19_"+col[0]+"_"+col[1])
df_output_seg_division_19_wide.columns=new_cols

In [57]:
df_output_seg_division_18=df_agg_POS_data_2018Q3_rewards_division.groupby(["segment","division_id","division_desc"])['sales','units','trans','customer_id_hashed'].agg(agg_function).reset_index()
df_output_seg_division_18_long=df_output_seg_division_18.rename(columns={"customer_id_hashed":"shopper_count"})
df_output_seg_division_18_wide=df_output_seg_division_18_long.pivot_table(index="segment",columns="division_desc",values=['sales','units','trans','shopper_count'])
df_output_seg_division_18_wide=df_output_seg_division_18_wide.reset_index()

new_cols=[]
for col in df_output_seg_division_18_wide.columns.tolist():
    if col[1]=="":
        new_cols.append(col[0])
    else:
        new_cols.append("Q3_18_"+col[0]+"_"+col[1])
df_output_seg_division_18_wide.columns=new_cols

In [60]:
df_output_seg_department_19_long=df_agg_POS_data_2019Q3_rewards_department.groupby(["segment","department_id","department_desc"])['sales','units','trans','customer_id_hashed'].agg(agg_function).reset_index()
df_output_seg_department_19_long=df_output_seg_department_19_long.rename(columns={"customer_id_hashed":"shopper_count"})

df_output_seg_department_18_long=df_agg_POS_data_2018Q3_rewards_department.groupby(["segment","department_id","department_desc"])['sales','units','trans','customer_id_hashed'].agg(agg_function).reset_index()
df_output_seg_department_18_long=df_output_seg_department_18_long.rename(columns={"customer_id_hashed":"shopper_count"})


In [61]:
df_output_seg_department_19_long.shape

(395, 7)

In [63]:
'''
def summarize_pivot_seg_product(df,product_id,product_col_desc,year):
    # df = input dateframe
    # product = "division_id" or "department_id"
    # product_col_desc = "division_desc" or "department_desc"
    # year = 4-digit interger 2019 or 2018
    df=df.groupby(["segment",product_id,product_col_desc])['sales','units','trans','customer_id_hashed'].sum(agg_function).reset_index()
    df=df.rename(columns={"customer_id_hashed":"shopper_count"})
    df=df.pivot_table(index="segment",columns=product_col_desc,values=['sales','units','trans','shopper_count'])
    df=df.reset_index()
    new_cols=[]
    for col in df.columns.tolist():
        if col[1]=="":
            new_cols.append(col[0])
        else:
            new_cols.append(col[0]+"_"+col[1])
    df.columns=new_cols
    

    for col in df.columns.tolist():
        if col!="segment":
            df=df.rename(columns={col:col+str(year)[:2]+"Q3"})
    return df
'''

'\ndef summarize_pivot_seg_product(df,product_id,product_col_desc,year):\n    # df = input dateframe\n    # product = "division_id" or "department_id"\n    # product_col_desc = "division_desc" or "department_desc"\n    # year = 4-digit interger 2019 or 2018\n    df=df.groupby(["segment",product_id,product_col_desc])[\'sales\',\'units\',\'trans\',\'customer_id_hashed\'].sum(agg_function).reset_index()\n    df=df.rename(columns={"customer_id_hashed":"shopper_count"})\n    df=df.pivot_table(index="segment",columns=product_col_desc,values=[\'sales\',\'units\',\'trans\',\'shopper_count\'])\n    df=df.reset_index()\n    new_cols=[]\n    for col in df.columns.tolist():\n        if col[1]=="":\n            new_cols.append(col[0])\n        else:\n            new_cols.append(col[0]+"_"+col[1])\n    df.columns=new_cols\n    \n\n    for col in df.columns.tolist():\n        if col!="segment":\n            df=df.rename(columns={col:col+str(year)[:2]+"Q3"})\n    return df\n'

In [64]:
df_output_seg_department_19_long=df_output_seg_department_19_long.fillna(0)
df_output_seg_department_18_long=df_output_seg_department_18_long.fillna(0)

df_output_seg_division_19_long=df_output_seg_division_19_long.fillna(0)
df_output_seg_division_18_long=df_output_seg_division_18_long.fillna(0)
df_output_seg_division_19_wide=df_output_seg_division_19_wide.fillna(0)
df_output_seg_division_18_wide=df_output_seg_division_18_wide.fillna(0)


In [65]:
writer=pd.ExcelWriter(write_folder+"/BL_output_AOV_analysis_by_segment_9_weeks_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output.to_excel(writer,"overall",index=False)

df_output_seg_division_19_wide.to_excel(writer,"divison_wide_19",index=False)
df_output_seg_division_18_wide.to_excel(writer,"divison_wide_18",index=False)

df_output_seg_division_19_long.to_excel(writer,"divison_long_19",index=False)
df_output_seg_division_18_long.to_excel(writer,"divison_long_18",index=False)

df_output_seg_department_19_long.to_excel(writer,"department_long_19",index=False)
df_output_seg_department_18_long.to_excel(writer,"department_long_18",index=False)

writer.save()